# Generador de personas aleatorias

In [1]:
import math
import random as rm
import numpy as np
import pandas as pd
import datetime as dt
import pytz
import re
from faker import Faker
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from funciones import quitar_acentos
from typing import Union

# Calcular CURP
def sacar_curp(nombre: str, apellido_paterno: str, apellido_materno: str, sexo: str, fecha_nac: Union[str, dt.datetime], estado_nac: str) -> str:
	nombre = quitar_acentos(nombre).upper().replace('Ñ', 'X')
	apellido_paterno = quitar_acentos(apellido_paterno).upper().replace('Ñ', 'X')
	apellido_materno = quitar_acentos(apellido_materno).upper().replace('Ñ', 'X')
	estado_nac = quitar_acentos(estado_nac.replace(' ', '_').lower())
	if isinstance(fecha_nac, str): fecha_nac = dt.datetime.strptime(fecha_nac, '%Y-%m-%d')
	# Primera letra y vocal del primer apellido
	curp = apellido_paterno[0:2].upper()
	# Primera letra del segundo apellido
	curp += apellido_materno[0:1].upper()
	# Primera letra del nombre de pila
	curp += nombre[0:1].upper()
	# Fecha de nacimiento (2 últimos dígitos del año, 2 del mes y 2 del día de nacimiento)
	curp += fecha_nac.strftime('%y%m%d')
	# Letra del sexo (H o M)
	curp += sexo[0:1].upper()
	# Dos letras correspondientes a la entidad de nacimiento segun RENAPO (Sonora => SR)
	# en el caso de extranjeros, se marca como NE (Nacido Extranjero)
	clave_estados = {
		'aguascalientes': 'AS', 'baja_california': 'BC', 'baja_california_sur': 'BS',
		'campeche': 'CC', 'coahuila': 'CL', 'colima': 'CM', 'chiapas': 'CS', 'chihuahua': 'CH',
		'distrito_federal': 'DF', 'ciudad_de_mexico': 'DF', 'durango': 'DG', 'guanajuato': 'GT',
		'guerrero': 'GR', 'hidalgo': 'HG', 'jalisco': 'JC', 'mexico': 'MC', 'estado_de_mexico': 'MC',
		'michoacan': 'MN', 'morelos': 'MS', 'nayarit': 'NT', 'nuevo_leon': 'NL', 'oaxaca': 'OC',
		'puebla': 'PL', 'queretaro': 'QT', 'quintana_roo': 'QR', 'san_luis_potosi': 'SP', 'sinaloa': 'SL',
		'sonora': 'SR', 'tabasco': 'TC', 'tamaulipas': 'TS', 'tlaxcala': 'TL', 'veracruz': 'VZ',
		'yucatan': 'YN', 'zacatecas': 'ZS'
	}
	if estado_nac in clave_estados:
		curp += clave_estados[estado_nac]
	else: curp += 'NE'
	# Primera consonante interna del primer apellido
	curp += re.sub(f'[aeiou]', '', apellido_paterno)[1:2].upper()
	# Primera consonante interna del segundo apellido
	curp += re.sub(f'[aeiou]', '', apellido_materno)[1:2].upper()
	# Primera consonante interna del nombre
	curp += re.sub(f'[aeiou]', '', nombre)[1:2].upper()
	# Dígito verificador del 0-9 para fechas de nacimiento hasta el año 1999 y A-Z para fechas de nacimiento a partir del 2000
	int_char = [rm.randint(0, 9), chr(rm.randint(65, 90))]
	curp += str(int_char[0] if fecha_nac.year <= 1999 else int_char[1])
	# Homoclave, para evitar duplicaciones
	curp += str(rm.randint(0, 9))
	return curp

# Instancia para usar servicio de geoubicaciones
geolocator = Nominatim(user_agent = 'geoapiExercises')
geocode = lambda query: geolocator.geocode(query, featuretype = 'city', language = 'es')
reverse = lambda coords: geolocator.reverse(coords, language = 'es')

# Obtener coordenadas del centro de una ciudad
def obtener_coordenadas(ciudad, estado, pais = 'Mexico') -> dict:
	print(f'\033[36mBuscando coordenadas de {ciudad} {estado}, {pais}\033[0m')
	try:
		# Hacer peticion a un api para obtener coordenadas del centro y extremas
		location = geocode(f'{ciudad} {estado} {pais}')
		# En caso de que no se haya encontrado nada
		if not location: raise BaseException(f'No se encontraron coordenadas de "{ciudad}", "{estado}", "{pais}"')
		location = location.raw
		# Obtener coordenadas del centro
		centro = (float(location['lat']), float(location['lon']))
		# Obtener extremos originales
		minx, maxx, miny, maxy = [float(i) for i in location['boundingbox']]
		ne, no, se, so = [(maxx, maxy), (maxx, miny), (minx, maxy), (minx, miny)]
		print(f'\033[33mobtener_coordenadas()\033[0m: Coordenadas encontradas en "{ciudad}", "{estado}", "{pais}"')
	except BaseException as e:
		print(f'\033[31mobtener_coordenadas()\033[0m:', e)
		return {}
	return {'centro': centro, 'noreste': ne, 'noroeste': no, 'sureste': se, 'suroeste': so}

# Generar n puntos random alrededor de un area
def generar_puntos_random(coords: dict, n = 1, reducir = True) -> list:
	# Ver si estan los 3 puntos necesarios
	coords_necesarias = ('centro', 'suroeste', 'noreste')
	if not all(k in coords for k in coords_necesarias):
		print('\033[33mgenerar_puntos_random()\033[0m: No se encontro alguna de las coordenadas necesarias:', coords_necesarias)
		return []
	# Obtener coordenadas del centro
	cenx, ceny = coords['centro']
	# Obtener extremos originales
	minx, miny = coords['suroeste']
	maxx, maxy = coords['noreste']
	# Hallar punto medio entre (suroeste centro) y (centro noreste)
	minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
	maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# True para reducir el area de la ciudad
	if reducir:
		# Hallar punto medio de puntos medios y centro
		minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
		maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# Obtener n puntos aleatorios dentro del area
	puntos_random = []
	while len(puntos_random) < n:
		puntos_random.append((rm.uniform(minx, maxx), rm.uniform(miny, maxy)))
	return puntos_random

# Buscar una direccion a partir de una lista de corrdenadas
def buscar_direccion(coords: dict) -> dict:
	# Generar puntos aleatorios alrededor del area
	puntos_random = generar_puntos_random(coords, 5)
	if not puntos_random: return {}
	dirrr = {}
	listo = False
	print('\033[36mBuscando direccion\033[0m')
	for punto_random in puntos_random:
		dirr = {}
		try:
			address = reverse(punto_random)
			if not address: continue
			direccion = address.raw['address']
			print(direccion)
			# Ver que campos se encontraron
			# https://nominatim.org/release-docs/latest/api/Output/#addressdetails
			# amenity, building, shop
			if any(k in direccion for k in ('amenity', 'building', 'shop', 'office', 'leisure')):
				if 'amenity' in direccion: dirr['local'] = direccion['amenity']
				elif 'building' in direccion: dirr['local'] = direccion['building']
				elif 'shop' in direccion: dirr['local'] = direccion['shop']
				elif 'office' in direccion: dirr['local'] = direccion['office']
				elif 'leisure' in direccion: dirr['local'] = direccion['leisure']
			# road
			if 'road' in direccion: dirr['calle'] = direccion['road']
			# house_number
			if 'house_number' in direccion: dirr['numero'] = direccion['house_number']
			else: dirr['numero'] = 'S/N'
			# neighbourhood, residential, suburb, quarter
			if any(k in direccion for k in ('neighbourhood', 'residential', 'suburb', 'quarter')):
				if 'neighbourhood' in direccion: dirr['colonia'] = direccion['neighbourhood']
				elif 'residential' in direccion: dirr['colonia'] = direccion['residential']
				elif 'suburb' in direccion: dirr['colonia'] = direccion['suburb']
				elif 'quarter' in direccion: dirr['colonia'] = direccion['quarter']
			# postcode
			if 'postcode' in direccion: dirr['cp'] = direccion['postcode']
			# city, town, village, hamlet
			if any(k in direccion for k in ('city', 'town', 'village', 'hamlet')):
				if 'city' in direccion: dirr['ciudad'] = direccion['city']
				elif 'town' in direccion: dirr['ciudad'] = direccion['town']
				elif 'village' in direccion: dirr['ciudad'] = direccion['village']
				elif 'hamlet' in direccion: dirr['ciudad'] = direccion['hamlet']
			# county
			if 'county' in direccion: dirr['municipio'] = direccion['county']
			# state
			if 'state' in direccion: dirr['estado'] = direccion['state']
			# country
			if 'country' in direccion: dirr['pais'] = direccion['country']
			# Si se encontro al menos calle y cp
			if all(k in dirr for k in ('calle', 'numero')): listo = True
			# Si ya se encontro una buena direccion, dejar de buscar
			if listo:
				print('\033[33mbuscar_direccion()\033[0m: Direccion encontrada en', punto_random)
				print('\033[32m', dirr, '\033[m')
				dirrr = dirr
				break
		except BaseException as e:
			print(f'\033[31mbuscar_direccion()\033[0m:', e)
			return {}
	if not listo: print('\033[33mbuscar_direccion()\033[0m: No se encontro una direccion precisa')
	return dirrr

# grados a radianes
def deg2rad(degrees): return math.pi * degrees / 180.0

# radianes a grados
def rad2deg(radians): return 180.0 * radians / math.pi

# Calcular radio de la tierra (R) en metros con una latitud dada segun WGS-84
def radio_terrestre_wgs84(lat: float) -> float:
	# http://en.wikipedia.org/wiki/Earth_radius
	# Semiejes de WGS-84
	semieje_mayor_a = 6378137.0  # Semieje mayor a [metros]
	semieje_menor_b = 6356752.31424  # Semieje menor b [metros]
	# Sacar R
	an = (semieje_mayor_a ** 2) * math.cos(lat)
	bn = (semieje_menor_b ** 2) * math.sin(lat)
	ad = semieje_mayor_a * math.cos(lat)
	bd = semieje_menor_b * math.sin(lat)
	return math.sqrt((an ** 2 + bn ** 2) / (ad ** 2 + bd ** 2))

# Calcular cuadro delimitador a partir de una cordenada dada y su amplitud
def bounding_box(coords: tuple, amplitud_en_km = 6) -> list:
	# convertir latitud y longitud a radianes
	lat_rad = deg2rad(coords[0])
	lon_rad = deg2rad(coords[1])
	# ampliar el area de busqueda
	amplitud = 1000 * amplitud_en_km

	# Radio terrestre en la latitud dada
	radius = radio_terrestre_wgs84(lat_rad)
	# Radio paralelo en la latitud
	pradius = radius * math.cos(lat_rad)

	# Calcular limites y convertirlos a grados
	lat_min = rad2deg(lat_rad - amplitud / radius)
	lon_min = rad2deg(lon_rad - amplitud / pradius)
	lat_max = rad2deg(lat_rad + amplitud / radius)
	lon_max = rad2deg(lon_rad + amplitud / pradius)
	return [(lat_min, lon_min), (lat_max, lon_max)]

In [2]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv', header = None)
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for i in range(n):
		print(f'\033[34mIdx{i}\033[0m')
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
		tipo_sangre = tipo_sangre[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()), maximum_age = 90)
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		if edad <= reap['edad'].max(): criter = (reap['edad'] == edad)
		else: criter = (reap['edad'] == reap['edad'].max())
		margen_ap = reap[criter].to_dict(orient = 'records')[0]

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'], margen_ap['lim_sup.altura.h.cm']), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'], margen_ap['lim_sup.peso.h.kg']), 2)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'], margen_ap['lim_sup.altura.m.cm']), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'], margen_ap['lim_sup.peso.m.kg']), 2)

		apellido_paterno = fake.last_name()
		apellido_materno = fake.last_name()

		id_estado = rm.choice(ct_ciudades['id_estado'].unique())
		ct_ciudad = ct_ciudades[ct_ciudades['id_estado'] == id_estado].sample()
		ct_ciudad_i_row = ct_ciudad.index.values[0]
		ct_ciudad = ct_ciudad.to_dict(orient = 'records')[0]

		ciudad = ct_ciudad['ciudad']
		estado = ct_ciudad['estado']
		curp = sacar_curp(nombre, apellido_paterno, apellido_materno, sexo, fecha_nac, estado)
		tel = f'({ct_ciudad["lada"]})' + ' {:03d} {:04d}'.format(rm.randint(0, 999), rm.randint(0, 9999))
		profesion = profesiones.sample().values[0][0] if edad >= 18 else None

		"""Correo"""
		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_paterno}',
			# alonso_perez
			3: f'{nombre}_{apellido_paterno}',
			# aps70
			4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_paterno}{apellido_materno}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)
		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos ahi mismo. El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		nss = None
		if profesion:
			# Los primeros dos caracteres están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			# suponiendo que la persona empezo a trabajar de sus 18 a sus 25
			nss2 = (fecha_nac + pd.DateOffset(months = rm.randint(216, 300))).strftime('%y')
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%04d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'

		"""Coordenadas de la ciudad"""
		centro = coordenadas = None
		# si el catalogo ya tiene coordenadas, tomar esos valores
		if pd.notna(ct_ciudad['centro']) and pd.notna(ct_ciudad['suroeste']) and pd.notna(ct_ciudad['noreste']):
			centro = eval(ct_ciudad['centro'])
			coordenadas = {
				'centro': centro,
				'suroeste': eval(ct_ciudad['suroeste']),
				'noreste': eval(ct_ciudad['noreste'])
			}
		else:
			# si no, buscar en internet el punto central y extremos segun la ciudad
			coordenadas = obtener_coordenadas(ciudad, estado)
			if coordenadas:
				centro = coordenadas['centro']
				# guardar coordenadas en el catalogo
				# para que a la otra no haya que buscar en internet
				ct_ciudades.loc[ct_ciudad_i_row, 'centro'] = str(centro)
				ct_ciudades.loc[ct_ciudad_i_row, 'suroeste'] = str(coordenadas['suroeste'])
				ct_ciudades.loc[ct_ciudad_i_row, 'noreste'] = str(coordenadas['noreste'])
				ct_ciudades.to_csv(file_ciudades, index = False)

		"""Direccion"""
		direcc = cp = None
		if centro and coordenadas:
			direccion = buscar_direccion(coordenadas)
			if direccion:
				cp = direccion['cp'] if 'cp' in direccion else None
				if all(k in direccion for k in ('calle', 'numero')):
					direcc = f"{direccion['calle']}, {direccion['numero']}"
					if 'colonia' in direccion: direcc += f", {direccion['colonia']}"

		"""Zona horaria"""
		timezone = tz_offset = None
		if centro:
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			#if centro[1] > 180.0 or centro[1] < -180.0 or centro[0] > 90.0 or centro[0] < -90.0:
			timezone = obj.timezone_at(lat = centro[0], lng = centro[1])
			# en Enero no hay DST
			#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')

		if not centro: centro = (np.nan, np.nan)

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_paterno': apellido_paterno,
			'apellido_materno': apellido_materno,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'tipo_sangre': tipo_sangre,
			'curp': curp,
			'correo': correo,
			'tel': tel,
			'color_fav': fake.safe_color_name(),
			'profesion': profesion,
			'nss': nss,
			'direccion': direcc,
			'cp': cp,
			'ciudad': ciudad,
			'estado': estado,
			'lat': centro[0],
			'lon': centro[1],
			'zona_horaria': timezone,
			'tz_offset': tz_offset,
		})

	return pd.DataFrame(personas)

personas_aleatorias(10)

Idx0
Buscando coordenadas de Tierranueva San Luis Potosí, Mexico
obtener_coordenadas(): No se encontraron coordenadas de "Tierranueva", "San Luis Potosí", "Mexico"
Idx1
Buscando direccion
{'road': 'Avenida la Pedrera', 'industrial': 'Ingenio "La Margarita"', 'town': 'Vicente Camalote', 'county': 'Acatlán de Pérez Figueroa', 'state': 'Oaxaca', 'country': 'México', 'country_code': 'mx'}
buscar_direccion(): Direccion encontrada en (18.50562366551575, -96.52242117807245)
 {'calle': 'Avenida la Pedrera', 'numero': 'S/N', 'ciudad': 'Vicente Camalote', 'municipio': 'Acatlán de Pérez Figueroa', 'estado': 'Oaxaca', 'pais': 'México'} 
Idx2
Buscando direccion
{'town': 'Isla de Cedros', 'county': 'Municipio de Ensenada', 'state': 'Baja California', 'postcode': '22990', 'country': 'México', 'country_code': 'mx'}
{'town': 'Isla de Cedros', 'county': 'Municipio de Ensenada', 'state': 'Baja California', 'postcode': '22990', 'country': 'México', 'country_code': 'mx'}
{'hamlet': 'Lomas Blanca', 'town': 

,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,altura,peso,tipo_sangre,curp,...,profesion,nss,direccion,cp,ciudad,estado,lat,lon,zona_horaria,tz_offset
0,Sra.,Barbara,Tapia,Soria,1965-11-25,M,169.29,48.82,A+,TASB651125MSPAOA43,...,Ingeniero de radiodifusión,91 83 65 7167 6,None,None,Tierranueva,San Luis Potosí,NaN,NaN,None,None
1,Dr.,Alberto,Serrato,Garica,1950-08-24,H,181.28,58.26,O+,SEGA500824HOCEAL06,...,Diseñador de cerámica,54 74 50 5836 0,"Avenida la Pedrera, S/N",None,Vicente Camalote,Oaxaca,18.513874,-96.525728,America/Mexico_City,-0500
2,Mtro.,Arturo,Saavedra,Colunga,1976-07-06,H,185.83,92.51,O+,SACA760706HBCAOR78,...,Librero,80 97 76 2374 2,None,None,Isla de Cedros,Baja California,28.096364,-115.188576,America/Tijuana,-0700
3,Ing.,Eric,Galarza,Mejía,1949-05-14,H,189.30,79.73,A+,GAME490514HTSAER94,...,Ingeniero automotriz,68 70 49 5100 0,None,None,Xicoténcatl,Tamaulipas,22.986414,-98.883693,America/Monterrey,-0500
4,None,Abel,Vigil,Paredes,1947-06-04,H,186.58,61.48,O+,VIPA470604HTLIAB39,...,Tecnólogo de ropa/textil,67 65 47 0702 7,"Calle Mariano Matamoros, S/N",None,Villa Mariano Matamoros,Tlaxcala,19.326944,-98.377222,America/Mexico_City,-0500
5,None,Mayte,Puga,Paredes,2004-05-24,M,155.99,76.53,O+,PUPM040524MSRUAAL5,...,None,None,None,None,Huepac,Sonora,29.912584,-110.212493,America/Hermosillo,-0700
6,None,Fabiola,Moya,Godínez,1954-01-30,M,174.60,65.12,O+,MOGF540130MNTOOA82,...,Voluntario,13 76 54 7401 7,"Laureles, S/N, Las Isabeles",63784,Xalisco,Nayarit,21.451602,-104.898804,America/Mazatlan,-0600
7,None,Servando,Roybal,Cordero,2010-09-25,H,150.93,28.34,A+,ROCS100925HGTOOES4,...,None,None,"Boulevard del Ferrocarril, S/N, Palo Verde",36947,Palo Verde,Guanajuato,20.324121,-101.851858,America/Mexico_City,-0500
8,None,Uriel,Riojas,Tijerina,1969-09-06,H,172.84,95.36,B-,RITU690906HBSIIR67,...,Taxista,29 94 69 3810 7,None,None,Las Lagunitas,Baja California Sur,NaN,NaN,None,None
9,None,Ilse,Barrientos,Navarrete,2012-08-16,M,138.80,27.77,O-,BANI120816MTLAALL7,...,None,None,None,None,Nanacamilpa,Tlaxcala,19.508032,-98.543241,America/Mexico_City,-0500


In [3]:
ciudades_perdidas = []
for i, ciudad in ct_ciudades.iterrows():
	# si esta ciudad no tiene estos tres campos
	no_hay_centro = pd.isna(ciudad['centro'])
	no_hay_suroeste = pd.isna(ciudad['suroeste'])
	no_hay_noreste = pd.isna(ciudad['noreste'])
	if (no_hay_centro and no_hay_suroeste and no_hay_noreste) or (no_hay_suroeste and no_hay_noreste):
		print(f'\033[34mIdx{i}\033[0m')
		# buscar en internet el punto central y extremos segun la ciudad
		coordenadas = obtener_coordenadas(ciudad['ciudad'], ciudad['estado'])
		if coordenadas:
			centro = coordenadas['centro']
			# guardar coordenadas en el catalogo
			# para que a la otra no haya que buscar en internet
			ct_ciudades.loc[i, 'centro'] = str(centro)
			ct_ciudades.loc[i, 'suroeste'] = str(coordenadas['suroeste'])
			ct_ciudades.loc[i, 'noreste'] = str(coordenadas['noreste'])
			ct_ciudades.to_csv(file_ciudades, index = False)
		else:
			ciudades_perdidas.append({
				'idx': i,
				'estado': ciudad['estado'],
				'cuidad': ciudad['ciudad']
			})
df = pd.DataFrame(ciudades_perdidas)
df.to_csv('./catalogos/ciudades_perdidas.csv', index = False)
print('\033[34mListo\033[0m')

Idx3
Buscando coordenadas de Ciénega Grande Aguascalientes, Mexico
obtener_coordenadas(): Coordenadas encontradas en "Ciénega Grande", "Aguascalientes", "Mexico"
Idx5
Buscando coordenadas de General Ignacio Zaragoza (Venadero) Aguascalientes, Mexico
obtener_coordenadas(): No se encontraron coordenadas de "General Ignacio Zaragoza (Venadero)", "Aguascalientes", "Mexico"
Idx6
Buscando coordenadas de General José María Morelos y Pavón (Cañada Honda) Aguascalientes, Mexico
obtener_coordenadas(): No se encontraron coordenadas de "General José María Morelos y Pavón (Cañada Honda)", "Aguascalientes", "Mexico"
Idx7
Buscando coordenadas de Jaltiche de Abajo Aguascalientes, Mexico
obtener_coordenadas(): No se encontraron coordenadas de "Jaltiche de Abajo", "Aguascalientes", "Mexico"
Idx11
Buscando coordenadas de La Punta Aguascalientes, Mexico
obtener_coordenadas(): No se encontraron coordenadas de "La Punta", "Aguascalientes", "Mexico"
Idx24
Buscando coordenadas de Alfredo V. Bonfil Baja Califo